# OHM Fork Extended Metrics
The definition used in this research for extended metrics is to cover anything that is calculated from the basic metrics already covered.  Each extended metric will be covered and will include definition, usage, and a code example.

The code below is used throughout the following examples.  It is done once here to reduce repetitiveness

In [33]:
from web3 import Web3
import requests
import math
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/40f886f8daa347ec901aa7e83d3300de')) # Infura

token_abi = open(f"abis/ohm.abi", "r").read()
stoken_abi = open(f"abis/sohm.abi", "r").read()
staking_abi = open(f"abis/sohmstaking.abi", "r").read()
supply_abi = open(f"abis/ohmcirculating.abi", "r").read()

token_contract = w3.eth.contract(Web3.toChecksumAddress('0x383518188c0c6d7730d91b2c03a03c837814a899'), abi=token_abi)
stoken_contract = w3.eth.contract(Web3.toChecksumAddress('0x04F2694C8fcee23e8Fd0dfEA1d4f5Bb8c352111F'), abi=stoken_abi)
staking_contract = w3.eth.contract(Web3.toChecksumAddress('0xFd31c7d00Ca47653c6Ce64Af53c1571f9C36566a'), abi=staking_abi)
supply_contract = w3.eth.contract(Web3.toChecksumAddress('0x0EFFf9199Aa1Ac3C3E34E957567C1BE8bF295034'), abi=supply_abi)

market_price = requests.get("https://api.coingecko.com/api/v3/simple/price?ids=olympus&vs_currencies=usd").json()['olympus']['usd']
market_price

439.18

## Staking Reward
The staking reward is to total OHM to be minted for this epoch.  Each epoch the staking reward calculated below is minted and this would be considered OHM inflation.  Many other extended metrics are based off of staking reward.

In [34]:
epoch = staking_contract.functions.epoch().call()
staking_reward = epoch[3] / math.pow(10, 9)
staking_reward

31053.0

## Staking Rebase
The staking rebase is the amount of newly minted OHM distributed per OHM staked.

In [35]:
circ = stoken_contract.functions.circulatingSupply().call() / math.pow(10, 9)
epoch = staking_contract.functions.epoch().call()

staking_reward = epoch[3] / math.pow(10, 9)
staking_rebase = staking_reward / circ
staking_rebase

0.005756330365148433

## Staking APY
This metric is somewhat obvious hopefully.  To get a value that matches with the Olympus DAO dashboard the staking_apy below needs to be multiplied by 100.

In [36]:
circ = stoken_contract.functions.circulatingSupply().call() / math.pow(10, 9)
epoch = staking_contract.functions.epoch().call()

staking_reward = epoch[3] / math.pow(10, 9)
staking_rebase = staking_reward / circ
staking_apy = math.pow(1 + staking_rebase, 365 * 3) - 1
staking_apy

535.5230215392438

## 5 Day ROI
The 5 day rate is a good metric to use when trying to determine whether to buy bonds which mature after 5 days or to simply buy and stake.  To get the % shown in the dashboard simply multiple by 100 similar to staking_apy above.

In [37]:
circ = stoken_contract.functions.circulatingSupply().call() / math.pow(10, 9)
epoch = staking_contract.functions.epoch().call()

staking_reward = epoch[3] / math.pow(10, 9)
staking_rebase = staking_reward / circ
five_day_rate = math.pow(1 + staking_rebase, 5 * 3) - 1
five_day_rate

0.08991246973810929

## Next Rebase
The block number for next rebase.

In [38]:
epoch = staking_contract.functions.epoch().call()
next_rebase = epoch[2]
next_rebase

13800600

## Staking TVL
Staking TVL is useful to compare to overall market cap.

In [39]:
circ = stoken_contract.functions.circulatingSupply().call() / math.pow(10, 9)
staking_tvl = circ * market_price
staking_tvl

2369192814.6740994

## Market Cap
The market capitalization is simply the current market price multiplied by the circulating supply.

In [40]:
circ = supply_contract.functions.OHMCirculatingSupply().call() / math.pow(10, 9)
market_cap = circ * market_price
market_cap

2677420781.689735

## Percent Staked
The percentage of OHM staked can be obtained by taking this extended metric and multiplying by 100.

In [41]:
staked_circ_supply = stoken_contract.functions.circulatingSupply().call() / math.pow(10, 9)
circ_supply = supply_contract.functions.OHMCirculatingSupply().call() / math.pow(10, 9)

percent_staked = staked_circ_supply / circ_supply
percent_staked

0.8848787724650768